In [31]:
# LSTM+CNN+Dropout
# trained on VAE and VGG16 feature data

# First, import necessary libraries
import tensorflow as tf
import numpy as np

In [32]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Conv1D
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [33]:
# uploading the X values from the autoencoder
X_input_auto = np.load('./auto_output01.npy')
X_input_VAE = X_input_auto[:,:212,:]
print(X_input_VAE.shape)
X_input_VGG = np.load('../Desktop/VGG16_feature_data.npy')
print(X_input_VGG.shape)
X_input = np.concatenate((X_input_VAE, X_input_VGG), axis=2)
print(X_input.shape)

(7, 212, 12288)
(7, 212, 4096)
(7, 212, 16384)


In [34]:
# uploading the Y values
y_data_input = fear_oneHot(212, 'fear_annotations_part01/MEDIAEVAL18_7_Fear.txt')
print(y_data_input.shape)

(212,)


In [35]:
# partition data into training and testing sets
train_num = 5
val_num = 2

X_train_data = X_input[:train_num, :, :]
X_valid_data = X_input[train_num:train_num+val_num, :, :]

print(X_train_data.shape)
print(X_valid_data.shape)

(5, 212, 16384)
(2, 212, 16384)


In [36]:
# uploading the y training data
timesteps = X_train_data.shape[1]
data_dim = X_train_data.shape[2]

# set up y_train_data master array
Y_train_data = np.zeros([train_num, timesteps])

# for each movie number between and including 7 and 13
for num in range(train_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_train_data[num, :] = y_data
print(Y_train_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [37]:
# upload the Y validation set

Y_valid_data = np.zeros([val_num, timesteps])

# for each movie number in validation set
for num in range(val_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+ train_num + num))
    # create a one-hot vector
    y_valid = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_valid_data[num, :] = y_valid
print(Y_valid_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [41]:
# model architecture
model = Sequential()

model.add(Conv1D(10, kernel_size=3, activation="sigmoid", input_shape=(timesteps, data_dim)))

model.add(LSTM(timesteps, return_sequences=True))
# input_shape=(timesteps, data_dim)
# dropout layer
model.add(Dropout(0.5))

# another LSTM layer
model.add(LSTM(timesteps, return_sequences=True))

# necessary flatten layer
model.add(Flatten()) 

# add the final dense layer and then softmax
model.add(Dense(timesteps, activation='softmax'))

In [42]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy'])

In [43]:
# running the LSTM model
model.fit(X_train_data, Y_train_data, epochs = 20, validation_data=(X_valid_data, Y_valid_data))
print("finished training!")

Train on 5 samples, validate on 2 samples
Epoch 1/20
5/5 [==============================] - 4s 715ms/step - loss: 0.9386 - binary_accuracy: 0.8255 - val_loss: 2.5915 - val_binary_accuracy: 0.7005
Epoch 2/20
5/5 [==============================] - 1s 234ms/step - loss: 0.9319 - binary_accuracy: 0.8255 - val_loss: 3.1845 - val_binary_accuracy: 0.7005
Epoch 3/20
5/5 [==============================] - 1s 261ms/step - loss: 2.5433 - binary_accuracy: 0.8255 - val_loss: 2.8892 - val_binary_accuracy: 0.7005
Epoch 4/20
5/5 [==============================] - 1s 238ms/step - loss: 1.6602 - binary_accuracy: 0.8255 - val_loss: 2.2245 - val_binary_accuracy: 0.7005
Epoch 5/20
5/5 [==============================] - 1s 284ms/step - loss: 1.0707 - binary_accuracy: 0.8255 - val_loss: 1.8602 - val_binary_accuracy: 0.7005
Epoch 6/20
5/5 [==============================] - 2s 326ms/step - loss: 1.0275 - binary_accuracy: 0.8255 - val_loss: 1.7146 - val_binary_accuracy: 0.7005
Epoch 7/20
5/5 [==================

In [ ]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[0])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

In [ ]:
#try visualizing this model at some point?